## 1. 选择模型

In [1]:
import getpass
import os

os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT_ID"] = "project-rag-1.0.0"
os.environ["LANGSMITH_PROJECT_NAME"] = "rag-sample"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"

from langchain_deepseek.chat_models import ChatDeepSeek  # 导入 DeepSeek 的聊天模型
from langchain_huggingface import HuggingFaceEmbeddings # 导入 HuggingFace 的嵌入模型
from langchain.chains import RetrievalQA # 导入检索问答链
from langchain.prompts import PromptTemplate # 导入提示模板
from langchain.chains import LLMChain # 导入 LLM 链
from langchain.memory import ConversationBufferMemory # 导入对话缓冲区内存
from langchain_chroma import Chroma # 导入 Chroma 向量存储

### 1. 聊天模型

In [2]:
# 创建 DeepSeek 聊天模型实例
deepseek_api_key = "sk-fffbb9b8a78d436a91a4780356b67a93"
# 选择deepseek-V3模型
llm = ChatDeepSeek(model="deepseek-chat", api_key = deepseek_api_key, temperature=0, base_url='https://api.deepseek.com')
# 选择deepseek-R1模型
# llm = ChatDeepSeek(model="deepseek-reason", api_key = deepseek_api_key, temperature=0, base_url='https://api.deepseek.com')

### 2. 嵌入模型

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 初始化模型
model_kwargs = {'device': 'cuda'} # 使用 GPU 进行推理
encode_kwargs = {'normalize_embeddings': True}  # 是否归一化

# 创建 HuggingFace 嵌入模型实例
# BGE系列的经典模型
bge_zh_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\zx\AppData\Local\Temp\ipykernel_36456\1769943512.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  bge_zh_embeddings = HuggingFaceEmbeddings(


In [4]:
# 从lier007/xiaobu-embedding-v2、dunzhang/stella-large-zh-v3-1792d 和 BAAI/bge-multilingual-gemma2 中蒸馏得到
ri_zh_embeddings = HuggingFaceEmbeddings(
    model_name="richinfoai/ritrieve_zh_v1",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [5]:
# BGE系列的新模型，参数更大，但速度更慢
bge_m3_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [6]:
# 也是中文的embedding模型
m3e_embeddings = HuggingFaceEmbeddings(
    model_name="moka-ai/m3e-base",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## 2. 构建索引

### 加载数据

#### JSONL文件

In [7]:
from langchain_community.document_loaders import JSONLoader


file_path = "database/json/all_data.jsonl" # 数据文件路径

# 定义元数据函数
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["title"] = record.get("Title")
    metadata["date"] = record.get("Date")
    metadata["viewcount"] = record.get("ViewCount")
    metadata['source'] = file_path

    return metadata


# 加载数据
json_loader = JSONLoader(
    file_path=file_path,
    jq_schema='.[]',
    content_key="Content",
    metadata_func=metadata_func,
    text_content=True,
    json_lines=True,
    )

json_data = json_loader.load()

In [8]:
# print(json_data)

#### PDF文件

In [9]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# 加载 PDF 文件
pdf_loader = PyPDFDirectoryLoader("database/pdf")
pdf_data = pdf_loader.load()

In [10]:
# print(pdf_data)

In [11]:
all_data = json_data + pdf_data

## 切分数据

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 创建文本分割器，设置分割参数
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)
# 将数据分割成小块
data_splits = text_splitter.split_documents(all_data)

## 存储数据

In [14]:
vectordb_ri = Chroma.from_documents(
    documents=data_splits, 
    embedding=ri_zh_embeddings,
    collection_name="ri_collection",
    persist_directory="ri_embedding",
)  # 创建 Chroma 向量存储实例

In [15]:
vectordb_bge = Chroma.from_documents(
    documents=data_splits, 
    embedding=bge_zh_embeddings,
    collection_name="bge_collection",
    persist_directory="bge_embedding",
)

## 3. 检索增强

### EnsembleRetriever（合并多个检索器）

In [16]:
from langchain_community.document_transformers import EmbeddingsClusteringFilter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import MergerRetriever

In [17]:
retriever_bge = vectordb_bge.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)
retriever_ri = vectordb_ri.as_retriever(
    search_type="mmr", search_kwargs={"k": 5}
)

lotr = MergerRetriever(retrievers=[retriever_bge, retriever_ri])

### 上下文压缩 + 重排序

In [32]:
from langchain_community.document_transformers import LongContextReorder

# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
reordering = LongContextReorder()
filter_by_retriever = EmbeddingsRedundantFilter(embeddings=bge_m3_embeddings)

pipeline = DocumentCompressorPipeline(transformers=[filter_by_retriever, reordering])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

### MultiQueryRetriever（多角度问题召回器）

In [33]:
import logging
from typing import List
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from langchain_core.runnables import chain

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines

def get_unique_ordered_docs(docs: List[Document]) -> List[Document]:
    """Get unique documents in the order they appear."""
    seen = set()
    unique_docs = []
    for doc in docs:
        if str(doc.page_content) not in seen:
            seen.add(str(doc.page_content))
            unique_docs.append(doc)
    return unique_docs

def multi_retriever(question: str) -> List[Document]:
    """Create a retriever that generates multiple queries."""
    # Prompt template for generating multiple queries
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""你的任务是生成三个不同的问题版本，以便从向量数据库中检索相关文档。通过从多个角度重新表述用户的问题，你的目标是帮助用户克服基于距离的相似性搜索的一些局限性。请将这些替代问题用换行分隔，不需要用数字或符号进行编号。首行生成原始问题，同样用换行分隔。原始问题：{question}""",
    )
    output_parser = LineListOutputParser()
    llm = ChatDeepSeek(model="deepseek-chat", api_key = deepseek_api_key, temperature=0.0, base_url='https://api.deepseek.com')
    llm_chain = QUERY_PROMPT | llm | output_parser
    retriever = MultiQueryRetriever(
        retriever=compression_retriever, 
        llm_chain=llm_chain, 
        parser_key="lines"
    )  # "lines" is the key (attribute name) of the parsed output
    docs = retriever.invoke(question)
    unique_docs = get_unique_ordered_docs(docs)
    
    return unique_docs


In [34]:
# question = "京剧旦角主要分为哪几个流派？"
# unique_docs = multi_retriever(question)
# Print the unique documents
# print(f"Number of unique documents: {len(unique_docs)}\n")
# print(unique_docs[0].page_content)
#print("=" * 50)
#print(unique_docs[-1].page_content)

### 自查询-过滤元数据

In [35]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="viewcount",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="date",
        description="The name of the movie director",
        type="string",
    ),
]

## 4. 生成回答

### 提示词模版

In [36]:
template = """使用以下的上下文片段回答最后的问题。
如果你不知道答案，只需说你不知道，不要编造答案。
答案最多使用三句话，并尽量保持简洁。在回答的最后总是说"谢谢提问！"

{context}

问题：{question}

回答：
"""

rag_prompt = PromptTemplate.from_template(template)

### LangGraph状态图

In [37]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = multi_retriever(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.invoke({"question": state["question"], "context": docs_content})
    llm = ChatDeepSeek(model="deepseek-chat", api_key = deepseek_api_key, base_url='https://api.deepseek.com')
    response = llm.invoke(messages)
    return {"answer": response.content}

In [38]:
from langgraph.graph import START, StateGraph

workflow = StateGraph(State).add_sequence([retrieve, generate])
workflow.add_edge(START, "retrieve")
app = workflow.compile()

### 主程序

In [39]:
# question = "京剧旦角主要分为哪几个流派？"

result = app.invoke({"question": "请详细阐述京剧的起源背景、形成过程及其在不同历史时期的发展演变，包括主要艺术特点、代表性人物和重要改革事件。"})

# print(f'Context: {result["context"]}\n\n')
print(f'Answer: \n{result["answer"]}')

INFO:langchain.retrievers.multi_query:Generated queries: ['京剧是如何从地方戏曲逐步融合演变成为国剧的？其关键历史节点和艺术融合过程是怎样的？', '京剧在不同朝代（如清代、民国、新中国）经历了哪些标志性变革？这些变革如何影响了其表演体系和艺术特征？', '从徽班进京到四大名旦时期，京剧艺术形式发生了哪些根本性转变？请分析唱腔、脸谱、行当等核心元素的历史流变及其文化内涵。']


Answer: 
京剧起源于清初流行于江南地区的徽班，1790年四大徽班进京为乾隆贺寿后留京发展，逐渐融合徽戏、秦腔、汉调等声腔，并吸收昆曲、京腔元素，于道光年间（1840-1860年）形成完整剧种。其艺术特点包括"唱念做打"四功、西皮二黄腔调及写意化舞台表现，代表人物有程长庚、谭鑫培、梅兰芳等，重要改革包括民国时期的"时装新戏"和新中国成立后的现代戏创作。谢谢提问！
